pyDPVis is a Python tool to visualize the structure of SAT instances and runs of the DPLL (Davis-Putnam-Logemann-Loveland) procedure. 

It uses networkx to display the problem's internal structure arising from its variable dependency graph. 

It can also generate animations showing dynamic change in structure during a DPLL algorithm run (wip).

This codebase includes a simplified DPLL in Python. Planning to add MiniSAT soon, as well as ability to visualize the search tree and clause learning.

I made this tool as an alternative to DPVis for teaching the DPLL algorithm, and for forming an intuitive sense between structure and hardness of SAT instances.

Daniel A. Espinosa Gonzalez 2024, Strukov Group at UC Santa Barbara
Work in progress ; Need to add faster visualization of very large SAT problems + better visualization of DPLL step by step

In [4]:
import graphmaker as gp
Xn, Yn, Zn, labels, color_mapping, node_colors, Xe, Ye, Ze = gp.make_graph_positions(filename='4_queens.txt')
gp.plot_graph_from_positions(Xn, Yn, Zn, labels, color_mapping, node_colors, Xe, Ye, Ze)

In [3]:
import plotly.graph_objects as go
import networkx as nx
from plotly.subplots import make_subplots
import rdimacs as rd

# CNF data here:
filename = '4_queens.txt'
num_vars, clauses = rd.read_dimacs(filename)

def generate_interaction_graph(num_vars, clauses):
    G = nx.Graph()
    for i in range(1, num_vars + 1):
        G.add_node(i)
        G.add_node(-i)
    for clause in clauses:
        for i in range(len(clause)):
            for j in range(i + 1, len(clause)):
                G.add_edge(clause[i], clause[j])
    return G

def create_frame(G, assignment, title):
    pos = nx.spring_layout(G)
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_color = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        if node in assignment:
            node_color.append('green')
        elif -node in assignment:
            node_color.append('red')
        else:
            node_color.append('blue')

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color=node_color,
            size=20,
            line_width=2),
        text=[str(node) for node in G.nodes()])

    return go.Frame(data=[edge_trace, node_trace], name=title)

def dpll(clauses, assignment=[]):
    if not clauses:
        return assignment
    if any([c == [] for c in clauses]):
        return False
    
    for clause in clauses:
        if len(clause) == 1:
            l = clause[0]
            new_clauses = [c for c in clauses if l not in c]
            new_clauses = [list(filter(lambda x: x != -l, c)) for c in new_clauses]
            return dpll(new_clauses, assignment + [l])
    
    l = clauses[0][0]
    new_clauses = [c for c in clauses if l not in c]
    new_clauses = [list(filter(lambda x: x != -l, c)) for c in new_clauses]
    result = dpll(new_clauses, assignment + [l])
    if result:
        return result
    return dpll(clauses, assignment + [-l])

def animate_dpll(num_vars, clauses, maxsteps=10):
    assignment = []
    steps = [(clauses, assignment)]
    frames = []
    timesteps = 0
    result = None

    while steps:
        current_clauses, current_assignment = steps.pop(0)
        result = dpll(current_clauses, current_assignment)
        if result is False:
            break
        new_assignment = current_assignment + result
        steps.append((current_clauses, new_assignment))
        G = generate_interaction_graph(num_vars, current_clauses)
        frame_title = f'Step {len(frames) + 1}'
        frames.append(create_frame(G, new_assignment, frame_title))
        timesteps += 1
        if timesteps > maxsteps:
            result = None
            break
    
    # Create initial figure
    initial_graph = generate_interaction_graph(num_vars, clauses)
    initial_frame = create_frame(initial_graph, [], 'Initial State')
    fig = go.Figure(data=initial_frame.data, layout=initial_frame.layout, frames=frames)

    # Add termination status
    if result is False:
        termination_message = "DPLL Terminated: UNSATISFIABLE"
    elif result is not None:
        termination_message = "DPLL Terminated: SATISFIABLE"
    else:
        termination_message = "DPLL Terminated: MAX STEPS REACHED"

    fig.update_layout(
        title=f"DPLL Algorithm Visualization<br>{termination_message}",
        updatemenus=[{
            'type': 'buttons',
            'showactive': False,
            'buttons': [
                {
                    'label': 'Play',
                    'method': 'animate',
                    'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True}]
                },
                {
                    'label': 'Pause',
                    'method': 'animate',
                    'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}]
                }
            ]
        }]
    )

    fig.show()

animate_dpll(num_vars, clauses, maxsteps=5)

# Is buggy, fix later:

# TODO: Make good data structures to handle the solution to DPLL over the problem ... then use that to update the graph
# You know, instead of solving DPLL per each step of the graph, which is ridiculous!

# Also, code DPLL from scratch in C and call it from Python.